# Gathering and cleaning data:
<br>
<br>The data holds information about flower species observed along with date, and location as well as pertinent
climate and environmental attributes, namely, day-length, daily temperatures, and precipitation. These are the 
variables that will be included in a flower phenology model.
<br>Five water years and early summers are included in the analysis starting in Fall of 2017 up to the current date, 
May 31, 2022. A water year in California runs from October through May the following year. Flowers at lower 
elevations bloom through summer.
<br>Years:  Oct 2016 : July 2016 / Oct 2017 : July 2017 / Oct 2018 : July 2018 / Oct 2019 : July 2019 / Oct 2020 : July 2021 / Oct 2021 : July 2022  
<br>Wildflower observation data comes from the iNaturalist API
<br>Climate data comes from Climate Explorer (NOAA data), hopefully an API will be used eventually. For now, data is downloaded and kept in a local directory.
<br>Day length information was calculated using the Skyfield API in Python.
          

______________________________

**Import the necessary libraries:**

In [1]:
!pip install pyinaturalist
!pip install pandas

from pyinaturalist.node_api import get_all_observations
import pandas as pd
import numpy as np
from datetime import date, datetime

print("Libraries imported!")

Libraries imported!


# Wildflower observations
**Import iNaturalist Observations using an API:**
<br> Use get_all_observations() from the the pyinaturalist.node_api (iNaturalist API)
to get plant species observation data for a place specified by place_id. Use taxon_id to specify taxonomic group.
<br>
<br>Sunol Regional Wilderness, CA (open space): place_id=3456
<br>Briones Regional Park, CA: place_id=3706
<br>Tilden Regional Park, CA: place_id=3523
<br>
<br>Flowering PLants: taxon_id = 47125 

In [2]:
#use today's date as the maximum date for data retreival
today = date.today()

In [3]:
#Call query through pyinaturalist.node_api, no authentication required

PLACES = [3456,3523,3706] #place_ids for east bay parks: sunol:3456, tilden:3523, briones:3706

Observations_EB = [] #initialize empty list for storing data

for p in PLACES:
    
    observations_research = get_all_observations(
      taxon_id=47125,             # Taxon ID for Flowering plants
      place_id=[p],               # Location ID from PLACES list
      d1='2017-09-30',            # Get observations from October 1st 2017...
      d2= today,                  # ...through today
      geo=True,                   # Only get observations with geospatial coordinates
      geoprivacy='open',          # Only get observations with public coordinates (not obscured/private)
      quality_grade = 'research'  #Only get research grade observations
    )
    print("Observations", str(p), "ready!", "There are:", len(observations_research), "observations")
    
    #add queried data, observations_research, to storage list, Observations_EB
    Observations_EB = Observations_EB+observations_research
    
print("Observations ready!","There are:", len(Observations_EB), "observations")

Observations 3456 ready! There are: 2803 observations


This request is larger than recommended for API usage. For bulk requests, consider using the iNat export tool instead: https://www.inaturalist.org/observations/export


Observations 3523 ready! There are: 5590 observations
Observations 3706 ready! There are: 3520 observations
Observations ready! There are: 11913 observations


In [5]:
#define function simplify_observation to extract the desired data from the query return list object, Observations_EB.
#returning a newly created dictionary of extracted values, simplified_obs.

def simplify_observation(obs):

    simplified_obs = {}
    
    # Top level values
    simplified_obs['Date'] = obs['observed_on']                                                        
    simplified_obs['Location_Name'] = obs['place_guess']
    simplified_obs['Place_ids'] =  obs['place_ids'] #find a way to do this based on query?
    simplified_obs['Region_Name'] = 'East Bay'
    simplified_obs['URL'] = obs['uri']
    simplified_obs['quality'] = obs['quality_grade']
                
    # Nested values
    simplified_obs['species_name'] = obs['taxon']['name']
    simplified_obs['coordinates'] = obs['geojson']['coordinates'] 
    simplified_obs['endemic'] = obs['taxon']['endemic']
    simplified_obs['native'] = obs['taxon']['native']
    simplified_obs['threatened'] = obs['taxon']['threatened']
    simplified_obs['threatened'] = obs['taxon']['observations_count']
    
#Name new columns and fill with values extracted from dataset
    
    #split lat, long into separate columns
    for i in range(len(obs['geojson']['coordinates'])):
        #create columns for lat and long separately
        simplified_obs['lat'] =obs['geojson']['coordinates'][0]
        simplified_obs['long'] =obs['geojson']['coordinates'][1]
    
    #split species name into separate columns, 'genus' and 'species'
    for i in range(len(obs['taxon']['name'])):
        tn = obs['taxon']['name'] + ' ' + 'none' + ' ' + 'none' #add a space to alleviate name anatomy issues
        #create columns for genus and species separately
        simplified_obs['genus'] =tn.split(' ')[0]
        simplified_obs['species'] =tn.split(' ')[1]
        simplified_obs['variety'] =tn.split(' ')[2]
    
    #Media/photo path columns
    # Range of values (photos)
    for i in range(len(obs['photos'])):        
        # Change value here if you want more or less than 3 photos 
        if(i<1):
            simplified_obs['photo '+str(i)] = obs['photos'][i]['url'].replace('square', 'original')   

    #create Place_ID column in df and fill with single value matching the iNat place_id, 
    #place_ids for east bay parks: sunol:3456, tilden:3523, briones:3706
    #for i in range(len(obs['place_ids'])):
        #d = obs['place_ids']
        #print(d)
        #[Place_ID.append(i) for i in PLACES if i in d] 
        #add column to df
        #simplified_obs['Place_ID'] = Place_ID[0]
    
    #create Place_ID column in df and fill with single value matching the iNat place_id, 
    #place_ids for east bay parks: sunol:3456, tilden:3523, briones:3706
    #for i in range(len(obs['place_ids'])):
        #d = obs['place_ids']
        #print(d)
        #[Place_ID.append(i) for p in PLACES if i == p] 
        #add column to df
        #simplified_obs['Place_ID'] = Place_ID[0]
    

    return simplified_obs

In [6]:
#apply function (simplify_observation() to each observation in Observations_EB 
#and store in list object simpleObs_EB)

simpleObs_EB = [simplify_observation(obs) for obs in Observations_EB]
print("Observations simplified!")

Observations simplified!


In [7]:
#convert list object simpleObs_EB to pandas dataframe df_EB
df_EB = pd.DataFrame.from_records(simpleObs_EB)
#df_EB.head(3)

In [8]:
#create Place_ID column to df and fill with single value matching the iNat place_id, 
#place_ids for east bay parks: sunol:3456, tilden:3523, briones:3706

PLACES = [3456,3523,3706] 
Place_ID = []

for d in df_EB['Place_ids']:
    #print(d)
    #print(type(d))
    #[print(i) for i in PLACES if i in d]
    [Place_ID.append(i) for i in PLACES if i in d]    

#add column to df
df_EB['Place_ID'] = Place_ID
print(df_EB['Place_ID'].unique())

[3456 3523 3706]


**Clean up the data set:**
<br>Add column 'park' to identify the park location of each observation.
<br>Drop columns: 'Location_Name' and 'Place_ids'

In [9]:
#Add park name to df based on the new 'Place_ID' column
# create a list of conditions
conditions = [
    (df_EB['Place_ID'] == 3456),
    (df_EB['Place_ID'] == 3523),
    (df_EB['Place_ID'] == 3706)
    ]

# create a list of the values we want to assign for each condition
values = ['Sunol', 'Tilden', 'Briones']

# create a new column and use np.select to assign values to it using our lists as arguments
df_EB['park'] = np.select(conditions, values)

# display updated DataFrame
#df_EB.head()

In [10]:
#drop 'Location_Name' and 'Place_ids' columns
df_EB.drop(['Location_Name','Place_ids'], axis=1, inplace=True)

#df_EB.head(3)

In [11]:
#dictionaries of tree and shrub genera and grass genera.
tree_shrub_dict = {'Salix':53453,'Cyperus':52734, 'Juglans':54495, 'Quercus':47851,'Acer':47727, 'Sambucus':52689,
                  'Populus':47566,'Schinus':57355, 'Platanus':49664, 'Toxicodendron':51079, 'Aesculus':53350, 
                  'Umbellularia':48810, 'Fraxinus':54806, 'Arbutus':51047, 'Alnus':53352, 'Lithocarpus':53956,
                  'Eucalyptus':51815, 'Prunus':47351}
                  
grass_dict = {'Carex':48571, 'Bromus':52701, 'Cortaderia':52715, 'Ehrharta':64143, 'Spartina':51826, 'Avena':52697,
              'Briza':57160}

#Combine the dictionaries of unwanted plants
not_included = dict(tree_shrub_dict)
not_included.update(grass_dict)

#create list of genus names to drop
not_included_names = list(not_included.keys())

print(not_included_names)

['Salix', 'Cyperus', 'Juglans', 'Quercus', 'Acer', 'Sambucus', 'Populus', 'Schinus', 'Platanus', 'Toxicodendron', 'Aesculus', 'Umbellularia', 'Fraxinus', 'Arbutus', 'Alnus', 'Lithocarpus', 'Eucalyptus', 'Prunus', 'Carex', 'Bromus', 'Cortaderia', 'Ehrharta', 'Spartina', 'Avena', 'Briza']


In [12]:
#Check the number of unique genera in dataframe df_EB
#print(df_EB['genus'].describe())

In [13]:
#drop tree, shrub, and grass genera
df_EB = df_EB[~df_EB['genus'].isin(not_included_names)]

In [14]:
#Check number of genera for reduction in rows and genera. 
#Some genera names in drop list may not occur in observations.
#print(df_EB['genus'].describe())

**Summary of data cleaning:**
<br>* Park name was added to df.
<br>* Plants in tree, shrub, or grass groups were dropped.
<br>* Dataframe has been reduced to 10513 rows and 332 genera of flowering plants.

_________________________________________
**Treat missing values:**

In [15]:
#check data types and look for columns with missing values
#df_EB.info()

#get counts by column for missing values using .isna().sum()
#print(df_EB.isna().sum())

#print(df_EB.shape)

**Desription of data and missing values:**
<br>There are 17 variables containing attributes for each plant observation. In total, for the three parks, 
<br>there are 10,434 observations of herbaceous, flowering plants. 
<br>All missing data (79 observations) are missing dates. We have no use for observations missing dates, 
<br>so we'll drop them.

In [16]:
#drop all observations with missing data
df_EB = df_EB.mask(df_EB.eq('None')).dropna()

#check to make sure no missing date vales left
print (df_EB.isna().sum()) 

Date            0
Region_Name     0
URL             0
quality         0
species_name    0
coordinates     0
endemic         0
native          0
threatened      0
lat             0
long            0
genus           0
species         0
variety         0
photo 0         0
Place_ID        0
park            0
dtype: int64


**Summary of missing data treatment:**
<br>* 79 rows with missing dates were dropped.
<br>* Plants in tree, shrub, or grass groups were dropped.
<br>* Dataframe has been reduced to 10434 rows and 331 genera of flowering plants.

______________________________
**Format Date:**
<br>Extract year, month and day. The various dataframes will be merged on the date columns.

In [17]:
#print(df_EB.head())

In [18]:
#Create a datetime-like value from DateTime

df_EB['DateTime'] = pd.to_datetime(df_EB['Date'], utc=True)
print(type(df_EB['DateTime']))

<class 'pandas.core.series.Series'>


In [19]:
#add columns for month and year
df_EB['Month'] = df_EB['DateTime'].dt.month
df_EB['Year'] = df_EB['DateTime'].dt.year
df_EB['Day'] = df_EB['DateTime'].dt.day

#df_EB.head(3)

In [20]:
#see place ids
#df_EB['Place_ID'].unique()

In [21]:
#reorder columns and select ones needed
df_EastBay = df_EB[['DateTime', 'Year', 'Month', 'Day', 'species_name', 'genus', 'species', 'quality',  
                    'park', 'Region_Name','lat', 'long', 'photo 0', 'URL']] 
#df_EastBay.head(3)

In [22]:
print(df_EastBay.shape)
col_names = df_EastBay.columns.values.tolist()
print(col_names)

(10494, 14)
['DateTime', 'Year', 'Month', 'Day', 'species_name', 'genus', 'species', 'quality', 'park', 'Region_Name', 'lat', 'long', 'photo 0', 'URL']


In [23]:
#see parks
#df_EastBay['park'].unique()


**Cleaned iNaturalist observations data desription:**
<br>Resulting dataframe df_EB has 10434 rows of observations, each containing information about a wildflower 
<br>observation uploaded to the iNaturalist app.
<br>The data frame contains 14 variables in columns: 
<br>   ['DateTime','Month','Year','Day','species_name','genus','species','quality',
<br>    'Location_Name','Region_Name','lat','long','photo 0','URL']

# Climate Data
**Import csv files**
<br>Temperature and precipitation data were downloaded from https://crt-climate-explorer.nemac.org, 
<br>managed by NOAA's Climate Program Office. Data is downloded separately for temperature and precipitation 
<br>from stations nearest to parks used in this tool.


In [24]:
import os
import pandas as pd
from datetime import datetime

__________________________________
**Import data:**
<br> - Retrieve .csv file data from local folder
<br> - As of june 6, 2022 these files cover up to date __________

In [25]:
## get current directory
folder_path = '/Users/sandidge/Desktop/Python_Projects/Springboard_coursework/Capstone2_Wildflowers/data_files/'

## list all file available 
all_files = os.listdir(folder_path)
#print(all_files)

## only store .csv filenames
csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
csv_files

## create a new list to store filesnames with no .csv extension
file_names = []
for x in range(len(csv_files)):
    file_names.append(csv_files[x].split('.')[0])


In [26]:
# get folder path
folder_path = '/Users/sandidge/Desktop/Python_Projects/Springboard_coursework/Capstone2_Wildflowers/data_files/'

df_names = []   #list to store df names produced
city_names = []          
station_id_codes = []
station_names = []

## Loop through to assign dataframe names
for file in file_names:
    final_df = file+"_df"
    #print("Dataframe name : "+final_df, type(final_df))
    df_names.append(final_df)      #add this one to list of df names 'df_names'
    city_name = file.split('_')[1].strip('Air')  #extract city name
    city_names.append(city_name)
    station_name = file.split('_')[1]  #extract station name
    station_names.append(station_name)
    station_id_code = file.split('_')[2].split('.')[0] #extract station id code
    station_id_codes.append(station_id_code) 
    
    filename = file+".csv"
    ## In python to assign a string as a dataframe name, use globals()
    globals()[final_df] = pd.read_csv(r'/Users/sandidge/Desktop/Python_Projects/Springboard_coursework/Capstone2_Wildflowers/data_files/'+filename)
    globals()[final_df]['city'] = city_name      #add column with city name
    globals()[final_df]['station_id'] = station_id_code     #add column wiht station id code
    #print(globals()[final_df])     #print the data frame
    

In [27]:
#create a dataframe with station and object name info
Stations = pd.DataFrame()
Stations['df_name'] = df_names
Stations['city_name'] = city_names
Stations['station_id_code'] = station_id_codes
Stations['station_names'] = station_names

Stations.sort_values(by = ['city_name'])

,df_name,city_name,station_id_code,station_names
2,tempF_Bakersfield_USC00040444_df,Bakersfield,USC00040444,Bakersfield
3,prec_Bakersfield_USC00040444_df,Bakersfield,USC00040444,Bakersfield
6,tempF_Concord_USW00023254_df,Concord,USW00023254,Concord
12,prec_Concord_USW00023254_df,Concord,USW00023254,Concord
4,tempF_HaywardAir_USW00093228_df,Hayward,USW00093228,HaywardAir
13,prec_HaywardAir_USW00093228_df,Hayward,USW00093228,HaywardAir
1,tempF_LivermoreAir_USW00023285_df,Livermore,USW00023285,LivermoreAir
9,prec_LivermoreAir_USW00023285_df,Livermore,USW00023285,LivermoreAir
7,prec_MtHamilton_USC00045933_df,MtHamilton,USC00045933,MtHamilton
11,tempF_MtHamilton_USC00045933_df,MtHamilton,USC00045933,MtHamilton


In [28]:
#merge precipitation data frame with temperature dataframe for each station

bakersfield = prec_Bakersfield_USC00040444_df.merge(tempF_Bakersfield_USC00040444_df, on = 'date', how = 'outer')
hayward = prec_HaywardAir_USW00093228_df.merge(tempF_HaywardAir_USW00093228_df, on = 'date', how = 'outer')
concord = prec_Concord_USW00023254_df.merge(tempF_Concord_USW00023254_df, on = 'date', how = 'outer')
livermore = prec_LivermoreAir_USW00023285_df.merge(tempF_LivermoreAir_USW00023285_df, on = 'date', how = 'outer')
mthamilton = prec_MtHamilton_USC00045933_df.merge(tempF_MtHamilton_USC00045933_df, on = 'date', how = 'outer')
oakland = prec_OaklandAir_USW00023230_df.merge(tempF_OaklandAir_USW00023230_df, on = 'date', how = 'outer')
sanjose = prec_SanJoseAir_USW00023293_df.merge(tempF_SanJoseAir_USW00023293_df, on = 'date', how = 'outer')
santamaria = prec_SantaMariaAir_USW00023273_df.merge(tempF_SantaMariaAir_USW00023273_df, on = 'date', how = 'outer')

In [29]:
#concatenate the city dataframes
climate_data = pd.concat([concord, oakland, hayward, livermore, sanjose, mthamilton])
print(climate_data.shape)

#verify concatenation
if int(len(climate_data)) == int(len(concord)+len(oakland)+len(hayward)+
                                 len(livermore)+len(sanjose)+len(mthamilton)):
    print('Concatenation seems correct!')


(135908, 11)
Concatenation seems correct!


In [30]:
climate_data.head(10)

,date,precipitation,precipitation_normal,city_x,station_id_x,min,max,normal_min,normal_max,city_y,station_id_y
0,19990606,0.00,10.34,Concord,USW00023254,51.0,76.0,56.1,82.0,Concord,USW00023254
1,19990607,0.00,10.35,Concord,USW00023254,48.0,72.0,56.3,82.3,Concord,USW00023254
2,19990608,0.00,10.36,Concord,USW00023254,49.0,75.0,56.4,82.5,Concord,USW00023254
3,19990609,T,10.36,Concord,USW00023254,50.0,78.0,56.5,82.8,Concord,USW00023254
4,19990610,T,10.37,Concord,USW00023254,52.0,81.0,56.6,83.0,Concord,USW00023254
5,19990611,T,10.38,Concord,USW00023254,53.0,77.0,56.8,83.3,Concord,USW00023254
6,19990612,T,10.38,Concord,USW00023254,53.0,80.0,56.9,83.5,Concord,USW00023254
7,19990613,T,10.39,Concord,USW00023254,52.0,83.0,57.0,83.7,Concord,USW00023254
8,19990614,T,10.39,Concord,USW00023254,55.0,78.0,57.1,84.0,Concord,USW00023254
9,19990615,T,10.40,Concord,USW00023254,54.0,74.0,57.2,84.2,Concord,USW00023254


__________________________
**Clean the data:**

<br>**Format Date:**
<br>Extract year, month and day. The various dataframes will be merged on the date columns.

In [31]:
climate_data['year_cl'] = climate_data['date'].astype(str).str[:4]
climate_data['month_cl'] = climate_data['date'].astype(str).str[4:6]
climate_data['day_cl'] = climate_data['date'].astype(str).str[6:8]

In [32]:
#get data for 2017 to present
climate_data = climate_data[(climate_data['year_cl']>= '2017') & (climate_data['year_cl']<'2022')]

print(climate_data.shape)

(10956, 14)


In [33]:
#print(climate_data.head())
climate_data.head(3)

,date,precipitation,precipitation_normal,city_x,station_id_x,min,max,normal_min,normal_max,city_y,station_id_y,year_cl,month_cl,day_cl
6419,20170101,T,0.11,Concord,USW00023254,42.0,55.0,40.4,57.1,Concord,USW00023254,2017,01,01
6420,20170102,0.16,0.22,Concord,USW00023254,42.0,53.0,40.4,57.1,Concord,USW00023254,2017,01,02
6421,20170103,0.69,0.32,Concord,USW00023254,46.0,58.0,40.5,57.1,Concord,USW00023254,2017,01,03


**Clean and rename columns:**

In [34]:
#drop unwanted columns, keep date, year, month, day, precipitation in inches, min daily temp, max daily temp
#station id number, and city
climate_data = climate_data[['year_cl', 'month_cl','day_cl', 'precipitation', 
                             'min', 'max', 'city_x', 'station_id_x']]
#rename temp cols
climate_data.rename(columns = {'min':'minTemp', 'max':'maxTemp', 'city_x':'city','station_id_x':'station_id'}, inplace = True)


In [35]:
climate_data.head()

,year_cl,month_cl,day_cl,precipitation,minTemp,maxTemp,city,station_id
6419,2017,01,01,T,42.0,55.0,Concord,USW00023254
6420,2017,01,02,0.16,42.0,53.0,Concord,USW00023254
6421,2017,01,03,0.69,46.0,58.0,Concord,USW00023254
6422,2017,01,04,1.40,47.0,58.0,Concord,USW00023254
6423,2017,01,05,1.40,34.0,52.0,Concord,USW00023254


In [36]:
climate_data.shape

(10956, 8)

_________________________________________
**Treat missing values:**

In [37]:
na_count = climate_data.isna().sum()
print(na_count)

year_cl           0
month_cl          0
day_cl            0
precipitation     0
minTemp          45
maxTemp          45
city              0
station_id        0
dtype: int64


In [38]:
#print the percentage of the total data that is missing for each col
#for i in na_count:
    #print(i,"is",((i/(climate_data.shape[0])*100),"percent of total observations"))

In [39]:
#look at the rows of missing data
climate_data_na = climate_data.loc[climate_data.isnull().any(axis=1)]
climate_data_na.head(3)

,year_cl,month_cl,day_cl,precipitation,minTemp,maxTemp,city,station_id
6634,2017,08,04,19.23,NaN,NaN,Concord,USW00023254
6642,2017,08,12,19.23,NaN,NaN,Concord,USW00023254
26040,2019,04,18,13.94,NaN,NaN,Oakland,USW00023230


In [40]:
#impute missing values in 'maxTemp' as average of nearest before date and after date
climate_data['maxTemp']=climate_data['maxTemp'].where(climate_data['maxTemp'].notnull(), 
                                          other=(climate_data['maxTemp'].fillna(method='ffill')
                                                 +climate_data['maxTemp'].fillna(method='bfill'))/2)

#impute missing values in 'minTemp' as average of nearest before date and after date
climate_data['minTemp']=climate_data['minTemp'].where(climate_data['minTemp'].notnull(), 
                                          other=(climate_data['minTemp'].fillna(method='ffill')
                                                 +climate_data['minTemp'].fillna(method='bfill'))/2)

In [41]:
na_count = climate_data.isna().sum()
print(na_count)

year_cl          0
month_cl         0
day_cl           0
precipitation    0
minTemp          0
maxTemp          0
city             0
station_id       0
dtype: int64


**Desription of data and missing values:**
<br>There are 8 variables containing attributes for each daily precipitation and temperatures observation. 
<br>In total, for all years, there are 10,956 observations. 
<br>All missing data (45 observations) are missing min and max temperatures. The number of missing values is less than 
<br>one percent of the total observation number. 
<br>Missing values are imputed as the average of the nearest value before and after the missing value(s).

# Get sunrise, sunset data from Skyfield and calculate day length:

In [42]:
! pip install skyfield

In [43]:
import skyfield
from skyfield import api
from skyfield import almanac
from datetime import date, datetime
import datetime
import pandas as pd

**lat, long coordinates for parks**
<br>Sunol Regional Wilderness = 37.510183, -121.82855
<br>Briones Regional Park= 37.935804, -122.137413
<br>Tilden Regional Park = 37.894647, -122.241635
<br>Mt. Diablo
<br>Henry Coe
<br>Carrizo

In [44]:
ts = api.load.timescale()
eph = api.load('de421.bsp')

#print(ts)
#print(eph)

**Get data from skyfield by looping over parks, coordinates to get all data sets**
<br>-

In [45]:
#create a dict of lat, long coords to loop over later
park_coords = {'Sunol': (37.510183, -121.82855),
'Briones': (37.935804, -122.137413), 
'Tilden': (37.894647, -122.241635)
}

#unpack dictionary as lists of parks and coordinate tuples
parks, coordinates = [list(x) for x in zip(*park_coords.items())]
print(parks)
print(coordinates)

['Sunol', 'Briones', 'Tilden']
[(37.510183, -121.82855), (37.935804, -122.137413), (37.894647, -122.241635)]


In [46]:
#use today's date as the maximum date for data retreival
today = date.today()
print(today)

2022-06-07


In [47]:
#full set of dates, remember to use date one day earlier and later than desired range
park_name = []
df_sun = pd.DataFrame(columns = ['timescale', 'sun', 'datetime','park_name'])

t0 = ts.utc(2017, 9, 30, 4) #off by 7 hourst0 = ts.utc(2017, 9, 30, 4) 
t1 = ts.now() #off by 7 hours

for c in coordinates:
    #print(c)
    print(c[0],c[1]) #print lat long of park as forst line in output
    
    #get data by using lat long for each park with the Skyfield api
    t, y = almanac.find_discrete(t0, t1, almanac.sunrise_sunset(eph, api.wgs84.latlon(c[0], c[1])))
    t_utc = t.utc_iso() #convert t (time) to utc_iso
    list_y = list(y)    #convert y (sun state 0,1) to a list for zipping
    
    #creat a dataframe: zip utc dates with 0, 1 sun state (sunrise, sunset)
    df = pd.DataFrame(zip(t_utc, list_y))
    
    if c[0] == coordinates[0][0]:
        #park_name.append('Sunol')
        park_NAME = 'Sunol'
    elif c[0] == coordinates[1][0]:
        #park_name.append('Briones')
        park_NAME = 'Briones'
    elif c[0] == coordinates[2][0]:
        #park_name.append('Tilden')
        park_NAME = 'Tilden'
    else:
        print('Park Name not matching')
    
    # create a list of datetime objects
    datetime = t.utc_datetime() #creates numpy array of datetime values
    #print(datetime[0],len(datetime), type(datetime))
    
    #create a datetime column
    df['datetime'] = datetime
    
    #create a list of len of zipped df and fill with the park name value
    name_list = [park_NAME] * (len(df))
    print(park_NAME, len(name_list), type(name_list))
    
    #create 'park_name' column
    df['park_name'] = name_list
    df.columns = ['timescale', 'sun', 'datetime','park_name']   
    
    #concatenate the df to the full df_sun
    df_sun = pd.concat([df_sun, df])

#print('')
#print("park df shape is:",df.shape)
#print(df.head(3))
print('')
print("Full df shape is:",df_sun.shape)
print(df_sun.head(3))


37.510183 -121.82855
Sunol 3424 <class 'list'>
37.935804 -122.137413
Briones 3424 <class 'list'>
37.894647 -122.241635
Tilden 3424 <class 'list'>

Full df shape is: (10272, 4)
              timescale sun                         datetime park_name
0  2017-09-30T14:02:17Z   1 2017-09-30 14:02:16.530649+00:00     Sunol
1  2017-10-01T01:51:23Z   0 2017-10-01 01:51:22.914560+00:00     Sunol
2  2017-10-01T14:03:09Z   1 2017-10-01 14:03:08.704979+00:00     Sunol


<br>**Convert dates and extract desired elements**
<br>
<br>

In [48]:
print(type(df_sun['timescale']))
print(type(df_sun['datetime']))
print(type(df_sun['datetime'][0]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [49]:
df_sun['datetime'] = pd.to_datetime(df_sun['datetime'],'%Y-%m-%d %H:%M')
df_sun.head()
#df_sun.info()

,timescale,sun,datetime,park_name
0,2017-09-30T14:02:17Z,1,2017-09-30 14:02:16.530649+00:00,Sunol
1,2017-10-01T01:51:23Z,0,2017-10-01 01:51:22.914560+00:00,Sunol
2,2017-10-01T14:03:09Z,1,2017-10-01 14:03:08.704979+00:00,Sunol
3,2017-10-02T01:49:52Z,0,2017-10-02 01:49:52.179909+00:00,Sunol
4,2017-10-02T14:04:01Z,1,2017-10-02 14:04:01.100832+00:00,Sunol


In [50]:
#create separate columns extracting year, month, day, hour, minute
df_sun['year'], df_sun['month'], df_sun['day'], df_sun['hour'], df_sun['minute'] =\
df_sun['datetime'].dt.year,\
df_sun['datetime'].dt.month,\
df_sun['datetime'].dt.day,\
df_sun['datetime'].dt.hour,\
df_sun['datetime'].dt.minute

In [51]:
#lat long coordinates for parks of interest

sunol_coords = api.wgs84.latlon(+37.510183, -121.82855)
briones_coords = api.wgs84.latlon(37.935804, -122.137413)
tilden_coords = api.wgs84.latlon(37.894647, -122.241635)

In [52]:
#this results in a time that is about a minute off 
#because the subtration of 7 hours would change the date 
# leading to a swap of sun position state (sunrise or sunset) codes
# here: 0 = sunset; 1 = sunrise
df_sun['hour_sub7'] = df_sun['hour'].replace({12:5, 0:17, 1:18, 2:19, 3:20, 4:21, 13:6, 14:7, 15:8})
print(df_sun[['datetime','sun','month','day','hour','minute','hour_sub7']].head(15))

                           datetime sun  month  day  hour  minute  hour_sub7
0  2017-09-30 14:02:16.530649+00:00   1      9   30    14       2          7
1  2017-10-01 01:51:22.914560+00:00   0     10    1     1      51         18
2  2017-10-01 14:03:08.704979+00:00   1     10    1    14       3          7
3  2017-10-02 01:49:52.179909+00:00   0     10    2     1      49         18
4  2017-10-02 14:04:01.100832+00:00   1     10    2    14       4          7
5  2017-10-03 01:48:21.809930+00:00   0     10    3     1      48         18
6  2017-10-03 14:04:53.731044+00:00   1     10    3    14       4          7
7  2017-10-04 01:46:51.836288+00:00   0     10    4     1      46         18
8  2017-10-04 14:05:46.609493+00:00   1     10    4    14       5          7
9  2017-10-05 01:45:22.292336+00:00   0     10    5     1      45         18
10 2017-10-05 14:06:39.750304+00:00   1     10    5    14       6          7
11 2017-10-06 01:43:53.210783+00:00   0     10    6     1      43         18

**Format date to YYYMMDD**
<br>This format will match the iNat and Cli-Mate data sets for merging

In [53]:
#single-digit days and months will not fit the 20220603 format, so add a leading zero to single digit date elements
z_months = []
# add 0 to one-digit days and months
for m in df_sun['month']:
    #print(m, type(m)) # m <class 'int'>
    i = str(m)
    ddigit = i.zfill(2)
    #df.replace(0, -1)
    #m.replace(int(ddigit))
    z_months.append(ddigit)
    
#print(z_months)
df_sun['Zmonth'] = z_months

z_days = []
# add 0 to one-digit days and months
for d in df_sun['day']:
    #print(d, type(d)) # d <class 'int'>
    j = str(d)
    ddigit = j.zfill(2)
    z_days.append(ddigit)
    
df_sun['Zday'] = z_days
#df_sun.head()
print("df_sun shape is:",df_sun.shape)



df_sun shape is: (10272, 12)


In [54]:
#combine year, month, day columns to get YYYMMDD format 'plain_date' that matches date format in other tables

plain_date = [] #empty string to hold returned values

# for every entry in df_sun: combine year, month, day into a date format 20220605 = YYYMMDD
for r in range(len(df_sun)):
    
    list_year = list(df_sun['year'])
    #print(len(list_year), type(list_year))

    list_month = list(df_sun['Zmonth'])
    #print(len(list_month), type(list_month))
    
    list_day = list(df_sun['Zday'])
    #print(len(list_day), type(list_day))
    
    date = int(str(list_year[r])+str(list_month[r])+str(list_day[r]))
    #print(type(date))
    plain_date.append(date)   #append date to the plain_date list
    
    
#print(len(plain_date), type(plain_date))

#add 'plain_date' column to df_sun dataframe
df_sun['plain_dates'] = plain_date
df_sun.head()

,timescale,sun,datetime,park_name,year,month,day,hour,minute,hour_sub7,Zmonth,Zday,plain_dates
0,2017-09-30T14:02:17Z,1,2017-09-30 14:02:16.530649+00:00,Sunol,2017,9,30,14,2,7,09,30,20170930
1,2017-10-01T01:51:23Z,0,2017-10-01 01:51:22.914560+00:00,Sunol,2017,10,1,1,51,18,10,01,20171001
2,2017-10-01T14:03:09Z,1,2017-10-01 14:03:08.704979+00:00,Sunol,2017,10,1,14,3,7,10,01,20171001
3,2017-10-02T01:49:52Z,0,2017-10-02 01:49:52.179909+00:00,Sunol,2017,10,2,1,49,18,10,02,20171002
4,2017-10-02T14:04:01Z,1,2017-10-02 14:04:01.100832+00:00,Sunol,2017,10,2,14,4,7,10,02,20171002


In [55]:
#use .pivot() to index on 'plain_dates' and create a single entry for each day 
#that holds both sunrise and sunset data.
df2=df_sun.pivot(index=['plain_dates', 'park_name'], columns=['sun'])
df2.head(3)

timescale 
 datetime 
 year 
 month 
 day 
 hour 
 minute 
 hour_sub7 
 Zmonth 
 Zday 
 
 
 
 sun 
 0 
 1 
 0 
 1 
 0 
 1 
 0 
 1 
 0 
 1 
 0 
 1 
 0 
 1 
 0 
 1 
 0 
 1 
 0 
 1 
 
 
 plain_dates 
 park_name 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20170930 
 Briones 
 NaN 
 2017-09-30T14:03:38Z 
 NaT 
 2017-09-30 14:03:37.860322+00:00 
 NaN 
 2017.0 
 NaN 
 9.0 
 NaN 
 30.0 
 NaN 
 14.0 
 NaN 
 3.0 
 NaN 
 7.0 
 NaN 
 09 
 NaN 
 30 
 
 
 Sunol 
 NaN 
 2017-09-30T14:02:17Z 
 NaT 
 2017-09-30 14:02:16.530649+00:00 
 NaN 
 2017.0 
 NaN 
 9.0 
 NaN 
 30.0 
 NaN 
 14.0 
 NaN 
 2.0 
 NaN 
 7.0 
 NaN 
 09 
 NaN 
 30 
 
 
 Tilden 
 NaN 
 2017-09-30T14:04:02Z 
 NaT 
 2017-09-30 14:04:02.193886+00:00 
 NaN 
 2017.0 
 NaN 
 9.0 
 NaN 
 30.0 
 NaN 
 14.0 
 NaN 
 4.0 
 NaN 
 7.0 
 NaN 
 09 
 NaN 
 30

In [56]:
#look at the multi-indexed columns
#df2.columns

In [57]:
#rename the columns as unique rather than levels
df2.columns = ['ts_set','ts_rise','dt_set', 'dt_rise', 'year_set', 'year_rise', 'month_set', 'month_rise',
             'day_set', 'day_rise','hour_set', 'hour_rise','minute_set', 'minute_rise',
              'hour_sub7_set', 'hour_sub7_rise', 'Zmonth_set', 'Zmonth_rise', 'Zday_set', 'Zday_rise']
df2.head(3)

ts_set 
 ts_rise 
 dt_set 
 dt_rise 
 year_set 
 year_rise 
 month_set 
 month_rise 
 day_set 
 day_rise 
 hour_set 
 hour_rise 
 minute_set 
 minute_rise 
 hour_sub7_set 
 hour_sub7_rise 
 Zmonth_set 
 Zmonth_rise 
 Zday_set 
 Zday_rise 
 
 
 plain_dates 
 park_name 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20170930 
 Briones 
 NaN 
 2017-09-30T14:03:38Z 
 NaT 
 2017-09-30 14:03:37.860322+00:00 
 NaN 
 2017.0 
 NaN 
 9.0 
 NaN 
 30.0 
 NaN 
 14.0 
 NaN 
 3.0 
 NaN 
 7.0 
 NaN 
 09 
 NaN 
 30 
 
 
 Sunol 
 NaN 
 2017-09-30T14:02:17Z 
 NaT 
 2017-09-30 14:02:16.530649+00:00 
 NaN 
 2017.0 
 NaN 
 9.0 
 NaN 
 30.0 
 NaN 
 14.0 
 NaN 
 2.0 
 NaN 
 7.0 
 NaN 
 09 
 NaN 
 30 
 
 
 Tilden 
 NaN 
 2017-09-30T14:04:02Z 
 NaT 
 2017-09-30 14:04:02.193886+00:00 
 NaN 
 2017.0 
 NaN 
 9.0 
 NaN 
 30.0 
 NaN 
 14.0 
 NaN 
 4.0 
 NaN 
 7.0 
 NaN 
 09 
 NaN 
 30

In [58]:
#reset the index to remove the milti-indexing and make 'plain_dates' and 'park_name' regular columns
df3 = df2.reset_index(level=['park_name','plain_dates'])
print(df3.columns)

#first three rows have nans because of time shift of 7 hours, these will be dropped
df3.head()

Index(['plain_dates', 'park_name', 'ts_set', 'ts_rise', 'dt_set', 'dt_rise',
       'year_set', 'year_rise', 'month_set', 'month_rise', 'day_set',
       'day_rise', 'hour_set', 'hour_rise', 'minute_set', 'minute_rise',
       'hour_sub7_set', 'hour_sub7_rise', 'Zmonth_set', 'Zmonth_rise',
       'Zday_set', 'Zday_rise'],
      dtype='object')


,plain_dates,park_name,ts_set,ts_rise,dt_set,dt_rise,year_set,year_rise,month_set,month_rise,...,hour_set,hour_rise,minute_set,minute_rise,hour_sub7_set,hour_sub7_rise,Zmonth_set,Zmonth_rise,Zday_set,Zday_rise
0,20170930,Briones,NaN,2017-09-30T14:03:38Z,NaT,2017-09-30 14:03:37.860322+00:00,NaN,2017.0,NaN,9.0,...,NaN,14.0,NaN,3.0,NaN,7.0,NaN,09,NaN,30
1,20170930,Sunol,NaN,2017-09-30T14:02:17Z,NaT,2017-09-30 14:02:16.530649+00:00,NaN,2017.0,NaN,9.0,...,NaN,14.0,NaN,2.0,NaN,7.0,NaN,09,NaN,30
2,20170930,Tilden,NaN,2017-09-30T14:04:02Z,NaT,2017-09-30 14:04:02.193886+00:00,NaN,2017.0,NaN,9.0,...,NaN,14.0,NaN,4.0,NaN,7.0,NaN,09,NaN,30
3,20171001,Briones,2017-10-01T01:52:29Z,2017-10-01T14:04:31Z,2017-10-01 01:52:29.271955+00:00,2017-10-01 14:04:31.145248+00:00,2017.0,2017.0,10.0,10.0,...,1.0,14.0,52.0,4.0,18.0,7.0,10,10,01,01
4,20171001,Sunol,2017-10-01T01:51:23Z,2017-10-01T14:03:09Z,2017-10-01 01:51:22.914560+00:00,2017-10-01 14:03:08.704979+00:00,2017.0,2017.0,10.0,10.0,...,1.0,14.0,51.0,3.0,18.0,7.0,10,10,01,01


**Missing values:**
<br> first and last date will have a missing sun state because the 7-hour offset truncates the first and last days
<br>as reversing the offset changes the date.
<br> Drop the first and last date

In [59]:
#look at first and last date, there are missing values
#df3.isna()
#sunrise times are missing for first date entry and sunset times are missing for last entry
df3.isna().sum()

In [60]:
#drop all observations with missing data
df3 = df3.mask(df3.eq('None')).dropna()
#df3.isna()         #first and last entries have been dropped
df3.isna().sum()    #there are no missing values
#df3.info()

**Calculate the day length:**
<br> hours and minutes between sunrise and sunset

In [61]:
#df3.columns

In [62]:
sr = list(df3.iloc[:,5])  #all rows, all elements : all columns, element 3 (datetime sunrise, sun = 1)
ss = list(df3.iloc[:,4])   #datetime sunset, sun = 0
#ss[5] #returns Timestamp('2017-10-02 01:51:23.272716+0000', tz='UTC') type object

In [63]:
#define timedelta functionto get day length in seconds
def delta(tset,trise):
    day_len = tset - trise
    py_day_len = day_len.to_pytimedelta() #convert to timedelta
    day_len_sec = py_day_len.seconds      #extract seconds value
    return (day_len_sec)

#test function
#delta(ss[4], sr[4])

In [64]:
#implement the function in a loop to get object holding day length value for each day
day_lengths = []
for r in range(len(df3)):
    tdelta = delta((ss[r]),(sr[r]))
    day_lengths.append(tdelta)  

#create 'day_length' column and fill with day_lengths
df3['day_length'] = day_lengths
df3.head(3)

,plain_dates,park_name,ts_set,ts_rise,dt_set,dt_rise,year_set,year_rise,month_set,month_rise,...,hour_rise,minute_set,minute_rise,hour_sub7_set,hour_sub7_rise,Zmonth_set,Zmonth_rise,Zday_set,Zday_rise,day_length
3,20171001,Briones,2017-10-01T01:52:29Z,2017-10-01T14:04:31Z,2017-10-01 01:52:29.271955+00:00,2017-10-01 14:04:31.145248+00:00,2017.0,2017.0,10.0,10.0,...,14.0,52.0,4.0,18.0,7.0,10,10,01,01,42478
4,20171001,Sunol,2017-10-01T01:51:23Z,2017-10-01T14:03:09Z,2017-10-01 01:51:22.914560+00:00,2017-10-01 14:03:08.704979+00:00,2017.0,2017.0,10.0,10.0,...,14.0,51.0,3.0,18.0,7.0,10,10,01,01,42494
5,20171001,Tilden,2017-10-01T01:52:55Z,2017-10-01T14:04:55Z,2017-10-01 01:52:55.005672+00:00,2017-10-01 14:04:55.370946+00:00,2017.0,2017.0,10.0,10.0,...,14.0,52.0,4.0,18.0,7.0,10,10,01,01,42479


In [65]:
#df3.columns

In [66]:
#remove unwanted columns
#df_daylength = df3[['plain_dates','park_name','year_set','Zmonth_set','Zday_set', 
#                    'hour_sub7_rise', 'minute_rise', 'hour_sub7_set', 'minute_set', 'day_length']]
#df_daylength.head(3)

In [67]:
#remove unwanted columns
df_daylength = df3[['plain_dates','park_name','year_set','month_set','day_set', 
                    'hour_sub7_rise', 'minute_rise', 'hour_sub7_set', 'minute_set', 'day_length']]
df_daylength.head(3)

,plain_dates,park_name,year_set,month_set,day_set,hour_sub7_rise,minute_rise,hour_sub7_set,minute_set,day_length
3,20171001,Briones,2017.0,10.0,1.0,7.0,4.0,18.0,52.0,42478
4,20171001,Sunol,2017.0,10.0,1.0,7.0,3.0,18.0,51.0,42494
5,20171001,Tilden,2017.0,10.0,1.0,7.0,4.0,18.0,52.0,42479


In [68]:
#rename columns
df_daylength.columns = [['plain_dates','park_name','year','month','day', 
                    'hour_rise', 'minute_rise', 'hour_set', 'minute_set', 'day_length']]
df_daylength.head(3)

,plain_dates,park_name,year,month,day,hour_rise,minute_rise,hour_set,minute_set,day_length
3,20171001,Briones,2017.0,10.0,1.0,7.0,4.0,18.0,52.0,42478
4,20171001,Sunol,2017.0,10.0,1.0,7.0,3.0,18.0,51.0,42494
5,20171001,Tilden,2017.0,10.0,1.0,7.0,4.0,18.0,52.0,42479


In [69]:
#create empty lists for storing values
yr = []
mn = []
dy = []
hrR = []
minR = []
hrS = []
minS = []

#convert from pd.DataFrame to pd.Series and then to a list
dl_year_list = list(df_daylength['year'].squeeze())
dl_month_list = list(df_daylength['month'].squeeze())
dl_day_list = list(df_daylength['day'].squeeze())
dl_hrR_list = list(df_daylength['hour_rise'].squeeze())
dl_minR_list = list(df_daylength['minute_rise'].squeeze())
dl_hrS_list = list(df_daylength['hour_set'].squeeze())
dl_minS_list = list(df_daylength['minute_set'].squeeze())

for r in range(len(df_daylength)):
    
    int_year = int(dl_year_list[r]) #convert each year value to an integer, then append
    yr.append(int_year)
    
    int_month = int(dl_month_list[r]) #convert each month value to an integer, then append
    mn.append(int_month)
    
    int_day = int(dl_day_list[r]) #convert each day value to an integer, then append
    dy.append(int_day)
    
    int_hour7_rise = int(dl_hrR_list[r])
    hrR.append(int_hour7_rise)
    
    int_minute_rise = int(dl_minR_list[r])
    minR.append(int_minute_rise)
    
    int_hour7_set = int(dl_hrS_list[r])
    hrS.append(int_hour7_set)
    
    int_minute_set = int(dl_minS_list[r])
    minS.append(int_minute_set)
    
df_daylength['year'] = yr
df_daylength['month'] = mn
df_daylength['day'] = dy
df_daylength['hour_rise'] = hrR
df_daylength['minute_rise'] = minR
df_daylength['hour_set'] = hrS
df_daylength['minute_set'] = minS

/var/folders/bv/4mctz8d94zx0nlh0hjsm4x8h0000gn/T/ipykernel_76399/549984427.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_daylength['year'] = yr
/var/folders/bv/4mctz8d94zx0nlh0hjsm4x8h0000gn/T/ipykernel_76399/549984427.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_daylength['month'] = mn
/var/folders/bv/4mctz8d94zx0nlh0hjsm4x8h0000gn/T/ipykernel_76399/549984427.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [70]:
df_daylength.head(15)

,plain_dates,park_name,year,month,day,hour_rise,minute_rise,hour_set,minute_set,day_length
3,20171001,Briones,2017,10,1,7,4,18,52,42478
4,20171001,Sunol,2017,10,1,7,3,18,51,42494
5,20171001,Tilden,2017,10,1,7,4,18,52,42479
6,20171002,Briones,2017,10,2,7,5,18,50,42332
7,20171002,Sunol,2017,10,2,7,4,18,49,42351
8,20171002,Tilden,2017,10,2,7,5,18,51,42334
9,20171003,Briones,2017,10,3,7,6,18,49,42187
10,20171003,Sunol,2017,10,3,7,4,18,48,42208
11,20171003,Tilden,2017,10,3,7,6,18,49,42189
12,20171004,Briones,2017,10,4,7,7,18,47,42042


In [71]:
#df_daylength.describe()

# Combine wildflower observations, climate data, and day length data


_________________________________________________
**merge the wildflower observation data frame, df_EastBay, with climate observation dataframe, climate_data**

In [74]:
#look at climate_data
climate_data.head(3)
#look at data types in climate_data
#climate_data.info()

,year_cl,month_cl,day_cl,precipitation,minTemp,maxTemp,city,station_id
6419,2017,1,1,T,42.0,55.0,Concord,USW00023254
6420,2017,1,2,0.16,42.0,53.0,Concord,USW00023254
6421,2017,1,3,0.69,46.0,58.0,Concord,USW00023254


In [75]:
#convert  <class 'pandas.core.series.Series'> date elements to strings then int
climate_data['year_cl'] = climate_data['year_cl'].astype(str).astype(int)
climate_data['month_cl'] = climate_data['month_cl'].astype(str).astype(int)
climate_data['day_cl'] = climate_data['day_cl'].astype(str).astype(int)

#climate_data.info()

In [76]:
df_EastBay.head(3) #look at the wildflower observation data

,DateTime,Year,Month,Day,species_name,genus,species,quality,park,Region_Name,lat,long,photo 0,URL
4,2017-10-03 21:48:55+00:00,2017,10,3,Rubus ursinus,Rubus,ursinus,research,Sunol,East Bay,-121.830453,37.516022,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246824
9,2017-10-03 21:51:32+00:00,2017,10,3,Artemisia douglasiana,Artemisia,douglasiana,research,Sunol,East Bay,-121.830070,37.515912,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246895
10,2017-10-03 22:01:18+00:00,2017,10,3,Croton setiger,Croton,setiger,research,Sunol,East Bay,-121.828453,37.514470,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246955


In [77]:
#df_EastBay.info() #check the data types for the columns you want to merge on

In [78]:
#Merge your files!
data_temp = pd.merge(df_EastBay,climate_data,left_on=['Year', 'Month', 'Day'], right_on=['year_cl','month_cl','day_cl'])
print('Files merged!')    

Files merged!


In [85]:
data_temp = pd.DataFrame(data_temp)
print(data_temp.shape)
data_temp.head(30) 

(50604, 22)


,DateTime,Year,Month,Day,species_name,genus,species,quality,park,Region_Name,...,photo 0,URL,year_cl,month_cl,day_cl,precipitation,minTemp,maxTemp,city,station_id
0,2017-10-03 21:48:55+00:00,2017,10,3,Rubus ursinus,Rubus,ursinus,research,Sunol,East Bay,...,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246824,2017,10,3,19.26,50.0,79.0,Concord,USW00023254
1,2017-10-03 21:48:55+00:00,2017,10,3,Rubus ursinus,Rubus,ursinus,research,Sunol,East Bay,...,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246824,2017,10,3,20.12,51.0,72.0,Oakland,USW00023230
2,2017-10-03 21:48:55+00:00,2017,10,3,Rubus ursinus,Rubus,ursinus,research,Sunol,East Bay,...,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246824,2017,10,3,13.69,49.0,72.0,Hayward,USW00093228
3,2017-10-03 21:48:55+00:00,2017,10,3,Rubus ursinus,Rubus,ursinus,research,Sunol,East Bay,...,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246824,2017,10,3,18.99,45.0,77.0,Livermore,USW00023285
4,2017-10-03 21:48:55+00:00,2017,10,3,Rubus ursinus,Rubus,ursinus,research,Sunol,East Bay,...,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246824,2017,10,3,14.08,49.0,74.0,SanJose,USW00023293
5,2017-10-03 21:48:55+00:00,2017,10,3,Rubus ursinus,Rubus,ursinus,research,Sunol,East Bay,...,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246824,2017,10,3,34.28,42.0,54.0,MtHamilton,USC00045933
6,2017-10-03 21:51:32+00:00,2017,10,3,Artemisia douglasiana,Artemisia,douglasiana,research,Sunol,East Bay,...,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246895,2017,10,3,19.26,50.0,79.0,Concord,USW00023254
7,2017-10-03 21:51:32+00:00,2017,10,3,Artemisia douglasiana,Artemisia,douglasiana,research,Sunol,East Bay,...,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246895,2017,10,3,20.12,51.0,72.0,Oakland,USW00023230
8,2017-10-03 21:51:32+00:00,2017,10,3,Artemisia douglasiana,Artemisia,douglasiana,research,Sunol,East Bay,...,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246895,2017,10,3,13.69,49.0,72.0,Hayward,USW00093228
9,2017-10-03 21:51:32+00:00,2017,10,3,Artemisia douglasiana,Artemisia,douglasiana,research,Sunol,East Bay,...,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246895,2017,10,3,18.99,45.0,77.0,Livermore,USW00023285


**Averaging weather station data to extimate temp and precipitation at each park**
<br>Parks are not associated with a single weather station. Often, multiple stations are similarly distant. A <br>geological feature such as a mountain range may change local weather and using averages from multiple stations 
<br>will help to account for this.
<br>A set of stations will be selected for each park and the climate data will be averaged. One record will remain <br>for each flower observation.

In [86]:
#check that all parks are there
#data['park'].unique()

In [87]:
#Add section that selects the stations to use for a given park and 
#get average temp and precipitation values across those stations
#will reduce dataset to 1/6 of current size

In [88]:
#produce a df with just one entry for each observation, using the averaged climate variable values for that park

_________________________________________________
**merge the data frame, data, with day length dataframe, df_daylength**

In [92]:
data_temp.head(3)

,DateTime,Year,Month,Day,species_name,genus,species,quality,park,Region_Name,...,photo 0,URL,year_cl,month_cl,day_cl,precipitation,minTemp,maxTemp,city,station_id
0,2017-10-03 21:48:55+00:00,2017,10,3,Rubus ursinus,Rubus,ursinus,research,Sunol,East Bay,...,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246824,2017,10,3,19.26,50.0,79.0,Concord,USW00023254
1,2017-10-03 21:48:55+00:00,2017,10,3,Rubus ursinus,Rubus,ursinus,research,Sunol,East Bay,...,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246824,2017,10,3,20.12,51.0,72.0,Oakland,USW00023230
2,2017-10-03 21:48:55+00:00,2017,10,3,Rubus ursinus,Rubus,ursinus,research,Sunol,East Bay,...,https://inaturalist-open-data.s3.amazonaws.com...,https://www.inaturalist.org/observations/8246824,2017,10,3,13.69,49.0,72.0,Hayward,USW00093228


In [93]:
data2 = data_temp[['DateTime','Year','Month', 'Day', 'species_name', 'genus','species', 'quality','park','Region_Name',
              'photo 0', 'URL', 'precipitation', 'minTemp', 'maxTemp', 'city', 'station_id']]

In [94]:
df_daylength.head(3)

,plain_dates,park_name,year,month,day,hour_rise,minute_rise,hour_set,minute_set,day_length
3,20171001,Briones,2017,10,1,7,4,18,52,42478
4,20171001,Sunol,2017,10,1,7,3,18,51,42494
5,20171001,Tilden,2017,10,1,7,4,18,52,42479


In [95]:
#issue merging multiplex dataframe day_length. Bypass by copying to clipboard with no index
data2.to_clipboard(index=False)
data2clip=pd.read_clipboard(sep='\t')

df_daylength.to_clipboard(index=False)
df_daylengthclip=pd.read_clipboard(sep='\t')

/var/folders/bv/4mctz8d94zx0nlh0hjsm4x8h0000gn/T/ipykernel_76399/3778243778.py:3: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  data2clip=pd.read_clipboard(sep='\t')


In [96]:
#Merge your files!
data = pd.merge(data2clip,df_daylengthclip,left_on=['Year', 'Month', 'Day'], right_on=['year','month','day'])
print('Files merged!') 

Files merged!


In [97]:
#Merge your files!
#data = pd.merge(data2,df_daylength,left_on=['Year', 'Month', 'Day'], right_on=['year','month','day'])
#print('Files merged!') 

In [98]:
data.head()

,DateTime,Year,Month,Day,species_name,genus,species,quality,park,Region_Name,...,plain_dates,park_name,year,month,day,hour_rise,minute_rise,hour_set,minute_set,day_length
0,2017-10-03 21:48:55+00:00,2017,10,3,Rubus ursinus,Rubus,ursinus,research,Sunol,East Bay,...,20171003,Briones,2017,10,3,7,6,18,49,42187
1,2017-10-03 21:48:55+00:00,2017,10,3,Rubus ursinus,Rubus,ursinus,research,Sunol,East Bay,...,20171003,Sunol,2017,10,3,7,4,18,48,42208
2,2017-10-03 21:48:55+00:00,2017,10,3,Rubus ursinus,Rubus,ursinus,research,Sunol,East Bay,...,20171003,Tilden,2017,10,3,7,6,18,49,42189
3,2017-10-03 21:48:55+00:00,2017,10,3,Rubus ursinus,Rubus,ursinus,research,Sunol,East Bay,...,20171003,Briones,2017,10,3,7,6,18,49,42187
4,2017-10-03 21:48:55+00:00,2017,10,3,Rubus ursinus,Rubus,ursinus,research,Sunol,East Bay,...,20171003,Sunol,2017,10,3,7,4,18,48,42208


**Final dataframe**
<br>The final data frame combines wildflower observations from iNaturalist, daily temperature and precipitation 
<br>from Climate Explorer, and sunset and sunrise times from Skyfield.
<br>This dataset includes observations from October 01, 2017 through May 24, 2022 and will be updated. 
<br>Data from Climate Explorer must be downloaded and kept in a local directory, while the others are acquired 
<br>through an API.
<br>The final dataframe includes 27 columns and 151,758 observations.
<br>
<br>**Next Steps**
<br>Currently each plant observation has 6 entries, one for each weather station. 
<br>Data from multiple stations will be averaged into one entry per plant observation.

In [100]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151812 entries, 0 to 151811
Data columns (total 27 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   DateTime       151812 non-null  object 
 1   Year           151812 non-null  int64  
 2   Month          151812 non-null  int64  
 3   Day            151812 non-null  int64  
 4   species_name   151812 non-null  object 
 5   genus          151812 non-null  object 
 6   species        151812 non-null  object 
 7   quality        151812 non-null  object 
 8   park           151812 non-null  object 
 9   Region_Name    151812 non-null  object 
 10  photo 0        151812 non-null  object 
 11  URL            151812 non-null  object 
 12  precipitation  151812 non-null  object 
 13  minTemp        151812 non-null  float64
 14  maxTemp        151812 non-null  float64
 15  city           151812 non-null  object 
 16  station_id     151812 non-null  object 
 17  plain_dates    151812 non-nul

In [102]:
#print(data.isna().sum())

DateTime         0
Year             0
Month            0
Day              0
species_name     0
genus            0
species          0
quality          0
park             0
Region_Name      0
photo 0          0
URL              0
precipitation    0
minTemp          0
maxTemp          0
city             0
station_id       0
plain_dates      0
park_name        0
year             0
month            0
day              0
hour_rise        0
minute_rise      0
hour_set         0
minute_set       0
day_length       0
dtype: int64


In [103]:
data.describe()

,Year,Month,Day,minTemp,maxTemp,plain_dates,year,month,day,hour_rise,minute_rise,hour_set,minute_set,day_length
count,151812.000000,151812.000000,151812.000000,151812.000000,151812.000000,1.518120e+05,151812.000000,151812.000000,151812.000000,151812.000000,151812.000000,151812.000000,151812.000000,151812.000000
mean,2019.716386,5.115604,15.972848,50.220487,70.622322,2.019769e+07,2019.716386,5.115604,15.972848,6.074381,29.290096,19.145917,33.694293,47321.295866
std,1.132845,2.275087,9.507880,7.611955,11.060399,1.128600e+04,1.132845,2.275087,9.507880,0.782842,17.742824,0.729944,17.383826,4747.419981
min,2017.000000,1.000000,1.000000,22.000000,29.000000,2.017100e+07,2017.000000,1.000000,1.000000,5.000000,0.000000,17.000000,0.000000,34285.000000
25%,2019.000000,4.000000,7.000000,46.000000,64.000000,2.019043e+07,2019.000000,4.000000,7.000000,6.000000,14.000000,19.000000,20.000000,44755.000000
50%,2020.000000,4.000000,17.000000,51.000000,70.000000,2.020042e+07,2020.000000,4.000000,17.000000,6.000000,27.000000,19.000000,34.000000,48731.000000
75%,2021.000000,6.000000,24.000000,55.000000,77.000000,2.021032e+07,2021.000000,6.000000,24.000000,6.000000,47.000000,20.000000,51.000000,50774.000000
max,2021.000000,12.000000,31.000000,85.000000,112.000000,2.021123e+07,2021.000000,12.000000,31.000000,8.000000,59.000000,20.000000,59.000000,53262.000000
